In [1]:
import pandas as pd
import psycopg2
import numpy as np
import re
from sqlalchemy import create_engine, text as sql_text
import datetime
from ydata_profiling import ProfileReport

c:\Users\Guilherme Bezerra\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Projeto Analytics Engineering - Inside Airbnb

## PostgreSQL - Exportação dos dados "raw"

Conjunto de dados:  "Inside Airbnb"

Passo a Passo:
1. Vá no link https://insideairbnb.com/get-the-data
2. Procure por "Rio de Janeiro"
3. Baixe os 3 .csv: "Listing", "Reviews" e Calendar" 

0bs: Caso queira pular estes passos, basta utilizar os .csvs que já estão nessa pasta: https://drive.google.com/drive/folders/1pwCpVsMu5si7BX1lzWX3tipP3fAPuoX3?usp=drive_link

4. Crie uma pasta dentro desse repositorio como 'raw_data' e adicione os 3 .csv

### Conexão com PostgreSQL

In [2]:
# James:
conn = psycopg2.connect(dbname='airbnb_project', user='postgres', host='localhost', password='admin', port="5432")

# Marcela: 
#conn = psycopg2.connect(dbname='myDBProject', user='postgres', host='localhost', password='0125698741', port="5433")
db_params = {
    'dbname': 'airbnb_project',
    'user': 'postgres',
    'host': 'localhost',
    'password': 'admin',
    'port': '5432'
}
db_url = f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}'


### Criação das bases 'raw's
- Crie um banco de dados PostgreSQL para armazenar os dados brutos das 3 tabelas ("Listing", "Reviews" e Calendar") 

In [3]:
engine = create_engine(db_url)
df_listings_raw = pd.read_csv("dbt/analytics_eng_airbnb/raw_data/listings.csv")
df_reviews_raw = pd.read_csv("dbt/analytics_eng_airbnb/raw_data/reviews.csv")
df_calendar_raw = pd.read_csv("dbt/analytics_eng_airbnb/raw_data/calendar.csv")

In [4]:
df_listings_raw.to_sql('listings_raw', engine, if_exists='replace', index=False)
df_reviews_raw.to_sql('reviews_raw', engine, if_exists='replace', index=False)
df_calendar_raw.to_sql('calendar_raw', engine, if_exists='replace', index=False)

595

In [13]:
def load_data_from_table(table_name):
    query = f"SELECT * FROM {table_name};"
    df = pd.read_sql_query(query, conn)
    return df

C:\Users\Guilherme Bezerra\AppData\Local\Temp\ipykernel_1308\1456954410.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [5]:
profile = ProfileReport(df_listings_raw.head(100), title="Listings Report")
profile.to_file("resultados_listings.html")
profile = ProfileReport(df_reviews_raw.head(100), title="Reviews Report")
profile.to_file("resultados_reviews.html")
profile = ProfileReport(df_calendar_raw.head(100), title="Calendar Report")
profile.to_file("resultados_calendar.html")

c:\Users\Guilherme Bezerra\AppData\Local\Programs\Python\Python312\Lib\site-packages\ydata_profiling\utils\dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)
Summarize dataset:  94%|█████████▍| 77/82 [00:01<00:00, 77.26it/s, Calculate auto correlation]                                    c:\Users\Guilherme Bezerra\AppData\Local\Programs\Python\Python312\Lib\site-packages\ydata_profiling\model\correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(i

MemoryError: Unable to allocate 8.11 TiB for an array with shape (1114150718243,) and data type float64

## DBT - Inicialização

Passo a Passo:
1) Instale o DBT
- python -m pip install dbt-postgres
- pip install dbt


2) Vá até sua pasta "dbt" e rode o comando:
3) dbt init analytics_eng_airbnb 

OBS: você pode pular os passos 2 e 3 utilizando a pasta "dbt" dentro do arquivo do Github


# Camadas

## Bronze

Verifique os arquivos criados na pasta dbt/analytics_eng_airbnb/models/bronze
- source.yml com os sources de "listing_raw", "reviews_raw", "calendar_raw"

- models.yml com informações e verificações sobre as novas tabelas que serão criadas: "listing_bronze", "reviews_bronze", "calendar_bronze". Nesse arquivo, executamos:
    - Verificamos se ID em "listings" é unique e not null
    - Adicionamos referencias de "foreign key" para listings_id

- arquivos .sql para cada uma das 3 tabelas

Para executar as regras criadas e importar as tabelas _bronze para o seu PostgreSQL, entre na pasta dbt/analytics_eng_airbnb e rode o comando:

- dbt run --models bronze

## Silver

Verifique os arquivos criados na pasta dbt/analytics_eng_airbnb/models/silver
- source.yml com os sources de "listing_bronze", "reviews_bronze", "calendar_bronze"

- models.yml com informações e verificações sobre as novas tabelas que serão criadas: "listing_silver", "reviews_silver", "calendar_silver". Nesse arquivo, executamos:
    - X Y Z (Aqui, fale sobre qualidade de dados: integridade, precisão e consistencia para os dados da camada). Fale também quais métricas foram selecionadas para continuar em cada lista e o porque. E também sobre os testes feitos (garantindo que a camada atenda as regras de negocio e requisitos de qualidade). Utilize as colunas selecionadas por Andre, explique porque escolheram elas e adicione os testes ">0" (e outros que tem lá).

- arquivos .sql para cada uma das 3 tabelas

Para executar as regras criadas e importar as tabelas _silver para o seu PostgreSQL, entre na pasta dbt/analytics_eng_airbnb e rode o comando:

- dbt run --models silver

## Gold

Verifique os arquivos criados na pasta dbt/analytics_eng_airbnb/models/gold
- source.yml com os sources de "listing_silver", "reviews_silver", "calendar_silver"

- models.yml com informações e verificações sobre as novas tabelas que serão criadas:  "listing_gold", "reviews_gold", "calendar_gold". Nesse arquivo, executamos:
    - X Y Z (Aqui, fale sobre agregações aplicadas - quais fora e porquê. Nessa camada testes não precisam ser executados (acho)).

- arquivos .sql para cada uma das 3 tabelas

Para executar as regras criadas e importar as tabelas _gold para o seu PostgreSQL, entre na pasta dbt/analytics_eng_airbnb e rode o comando:

- dbt run --models gold